In [9]:
import os
from openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = ""
job_id = 'ftjob-hjLV1XIM6ccPjzx6zTpAYw5z'

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
finetuned_job = client.fine_tuning.jobs.retrieve(job_id)
finetuned_model = finetuned_job.fine_tuned_model

In [10]:

loader = PyPDFLoader("../_RAG_3.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)
db.save_local("_FAISS")

In [11]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A90791B5C0>)

In [12]:
prompt = """Your name is  Assistant. You are a personal assistant . Recommend ideal  exercises to help users to improve their  skills.
    You help users to learn  fundamentals and techniques with detailed answers. You also enlight users about  products. Give answers with 2 or 3 sentences, not longer.
Context: {context}
Question: {question} 
"""

custom_rag_prompt = PromptTemplate.from_template(prompt)
retriever = db.as_retriever()
llm = ChatOpenAI(model_name=finetuned_model, temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [13]:

rag_chain.invoke("what is  x?")


" X is the outdoor putting practice solution from . It uses Microsoft's HoloLens Augmented Reality technology to display valuable putt information as an overlay onto the green. It can be used with a smartphone app for coaching scenarios and works on any green in the world."